In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import anndata

/Users/mqp/miniconda3/envs/scanpy/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
# Read anndata objects. Assume working directory to be 'becape/docs/'
dir_baron = "./anndata/baron.h5ad"
adata_baron = sc.read(dir_baron)

dir_segerstolpe = "./anndata/segerstolpe.h5ad"
adata_segerstolpe = sc.read(dir_segerstolpe)

In [3]:
# Conversion to R changes cell annotations to factors 
# -> export cell type annotations to be later rewritten into the R obj
# here we are selecting ['dblabel'] as the annotation column. Modify this for the desired annotation column

baron_annot = adata_baron.obs['dblabel']
baron_annot.to_csv('./anndata/baron_annot.csv', index=False, header=False)

segerstolpe_annot = adata_segerstolpe.obs['dblabel']
segerstolpe_annot.to_csv('./anndata/segerstolpe_annot.csv', index=False, header=False)

In [4]:
# For SCDC and MuSiC we want expressions stored in AnnData.raw us they contain all sequenced genes, 
# not just highly variable genes filtered further downstream of the BESCA workflow
obs = adata_baron.obs
var = adata_baron.raw.var
uns = adata_baron.uns
raw = adata_baron.raw
adata_baron_raw = anndata.AnnData(raw.X, obs=obs, var=var, uns=uns, raw=raw)

obs = adata_segerstolpe.obs
var = adata_segerstolpe.raw.var
uns = adata_segerstolpe.uns
raw = adata_segerstolpe.raw
adata_segerstolpe_raw = anndata.AnnData(raw.X, obs=obs, var=var, uns=uns, raw=raw)

### Fix segerstolpe index - needs to contain patient id
This is required by the BisqueRNA::SeuratToExpressionSet() R package: https://rdrr.io/cran/BisqueRNA/man/SeuratToExpressionSet.html.  
The sample index has to contain subscriptable individual patient IDs.

Example correct patient ID: **ERR1630619_T2D1**, where ERR1630619 is the cell UMI and T2D1 is the unique patient ID we want.  
We will then specify this in the R script that calls the `BisqueRNA::SeuratToExpressionSet()` method

`BisqueRNA::SeuratToExpressionSet(seurat.object, delimiter='_', position='2', version = "v3")`  
`delimiter` = Character to split cell names with to find individual ID.   
`position` = Integer indicating 1-indexed position of individual ID after splitting cell name with delimiter. R indexing starts from 1


In [5]:
idx = adata_segerstolpe_raw.obs.index
idx_new = []
for i, val in enumerate(idx):
    out = val + '_' +adata_segerstolpe_raw.obs['Sample Characteristic[individual]'][i]
    idx_new.append(out)
    
adata_segerstolpe_raw.obs.index = idx_new
adata_segerstolpe_raw.obs.head()

,Sample Characteristic[organism],Sample Characteristic Ontology Term[organism],Sample Characteristic[individual],Sample Characteristic Ontology Term[individual],Sample Characteristic[sex],Sample Characteristic Ontology Term[sex],Sample Characteristic[age],Sample Characteristic Ontology Term[age],Sample Characteristic[body mass index],Sample Characteristic Ontology Term[body mass index],...,score_ILC3_scanpy,score_ActTcell_scanpy,score_NaiTcell_scanpy,score_McellGut_scanpy,score_Mast_scanpy,celltype0,celltype1,celltype2,celltype3,dblabel
ERR1630619_T2D1,Homo sapiens,http://purl.obolibrary.org/obo/NCBITaxon_9606,T2D1,NaN,male,http://purl.obolibrary.org/obo/PATO_0000384,57 year,NaN,24 kilogram per square meter,NaN,...,0.076371,0.121068,-0.011084,-0.011230,-0.073558,epithelial cell,enteroendocrine cell,pancreatic A cell,pancreatic A cell,pancreatic A cell
ERR1630620_T2D1,Homo sapiens,http://purl.obolibrary.org/obo/NCBITaxon_9606,T2D1,NaN,male,http://purl.obolibrary.org/obo/PATO_0000384,57 year,NaN,24 kilogram per square meter,NaN,...,-0.085040,0.289098,-0.003337,-0.056040,0.064756,epithelial cell,enteroendocrine cell,pancreatic A cell,pancreatic A cell,pancreatic A cell
ERR1630621_T2D1,Homo sapiens,http://purl.obolibrary.org/obo/NCBITaxon_9606,T2D1,NaN,male,http://purl.obolibrary.org/obo/PATO_0000384,57 year,NaN,24 kilogram per square meter,NaN,...,-0.059220,-0.043709,-0.006920,-0.046709,-0.120627,epithelial cell,enteroendocrine cell,pancreatic A cell,pancreatic A cell,pancreatic A cell
ERR1630622_T2D1,Homo sapiens,http://purl.obolibrary.org/obo/NCBITaxon_9606,T2D1,NaN,male,http://purl.obolibrary.org/obo/PATO_0000384,57 year,NaN,24 kilogram per square meter,NaN,...,-0.086758,-0.365977,-0.013570,0.144259,0.112262,epithelial cell,enteroendocrine cell,PP cell,PP cell,PP cell
ERR1630623_T2D1,Homo sapiens,http://purl.obolibrary.org/obo/NCBITaxon_9606,T2D1,NaN,male,http://purl.obolibrary.org/obo/PATO_0000384,57 year,NaN,24 kilogram per square meter,NaN,...,0.020336,-0.054373,-0.032912,0.222236,-0.253834,epithelial cell,enteroendocrine cell,PP cell,PP cell,PP cell


### Convert to SingleCellExperiment R Object

saves anndata as SCE object. Postprocess in R

based on tutorial here: https://github.com/LuckyMD/Code_snippets/blob/master/Seurat_to_anndata.ipynb

These `%%R` magic functions allow us to run R from jupyter cells. The conversion R script is also provided in the 'bescape/docs/sce_to_eset.R'. For large datasets, one might need to run the script from the command line to allocate more memory as follows:

`$ env R_MAX_VSIZE=100Gb R`

then from R shell:  
`source('sce_to_eset.R')`


Note: To convert from SCE to ESET, we need Seurat version  3.0.2, later versions contain library(reticulate) will cause jupyter to crash:
https://stackoverflow.com/questions/58611486/r-kernel-crashes-while-loading-r-package-using-rpy2

In [3]:
import anndata2ri
anndata2ri.activate()

In [4]:
%load_ext rpy2.ipython

First convert anndata to sce

In [39]:
%%R -i adata_baron_raw
saveRDS(adata_baron_raw, './anndata/baron_raw_sce.RDS')

In [37]:
%%R -i adata_segerstolpe_raw
saveRDS(adata_segerstolpe_raw, './anndata/segerstolpe_raw_sce.RDS')

Convert SCE to eset

In [ ]:
%%R 
library(Seurat)
library(BisqueRNA)

sce_to_seurat <- function(sce_path, sc_anno_path, filename='eset.RDS'){
    sce <- readRDS(sce_path)
    sc_anno <- read.csv(sc_anno_path, header=FALSE)

    seurat <- as.Seurat(sce, counts=NULL, data='X')

# TODO add celltype_dream as param
    seurat@meta.data$celltype_dream <- sc_anno$V1
    Idents(seurat)<-"celltype_dream"
    seurat@assays$RNA@counts<-seurat@assays$RNA@data
    if(!is.null(filename)){
        saveRDS(seurat, file=filename)
    }
    return(seurat)

}

seurat_to_eset <- function(seurat, delim='-', idx=2, filename='seurat.RDS'){
    # Idents(liver_h5ad) %>% head() # to check sample names
    out.eset <- BisqueRNA::SeuratToExpressionSet(seurat, delim, idx, version = 'v3')
    # fix GeneID
    out.eset@featureData@data$GeneID <- featureNames(out.eset)
    #assign cluster to just characters
    out.eset@phenoData@data$cluster<-as.character(out.eset@phenoData@data$cellType)
    #update vardata
    varMetadata(out.eset)<-data.frame(labelDescription=colnames(out.eset@phenoData@data),stringsAsFactors=FALSE)

    out.eset@featureData@varMetadata<-data.frame(labelDescription="GeneID",stringsAsFactors=FALSE)

    rownames(out.eset@featureData@varMetadata)<-"GeneID"

    saveRDS(out.eset, file=filename)
    return(out.eset)
}


seurat <- sce_to_seurat(sce_path='./anndata/segerstolpe_raw_sce.RDS', 
                        sc_anno_path='./anndata/segerstolpe_annot.csv', 
                        filename=NULL)
eset <- seurat_to_eset(seurat, delim='_', idx=2, filename='./anndata/segerstolpe_raw_eset.RDS')


R[write to console]: 
 *** caught segfault ***

R[write to console]: address 0x10, cause 'memory not mapped'

R[write to console]: 
Traceback:

R[write to console]:  1: 
R[write to console]: py_initialize(config$python, config$libpython, config$pythonhome, 
R[write to console]:     config$virtualenv_activate, config$version >= "3.0", interactive(), 
R[write to console]:     numpy_load_error)
R[write to console]: 

R[write to console]:  2: 
R[write to console]: doTryCatch(return(expr), name, parentenv, handler)
R[write to console]: 

R[write to console]:  3: 
R[write to console]: tryCatchOne(expr, names, parentenv, handlers[[1L]])
R[write to console]: 

R[write to console]:  4: 
R[write to console]: tryCatchList(expr, classes, parentenv, handlers)
R[write to console]: 

R[write to console]:  5: 
R[write to console]: tryCatch({
R[write to console]:     py_initialize(config$python, config$libpython, config$pythonhome, 
R[write to console]:         config$virtualenv_activate, config$versio

R[write to console]:         Idents(seurat) <- "celltype_dream"
R[write to console]:         seurat@assays$RNA@counts <- seurat@assays$RNA@data
R[write to console]:         if (!is.null(filename)) {
R[write to console]:             saveRDS(seurat, file = filename)
R[write to console]:         }
R[write to console]:         return(seurat)
R[write to console]:     }
R[write to console]:     seurat_to_eset <- function(seurat, delim = "-", idx = 2, 
R[write to console]:         filename = "seurat.RDS") {
R[write to console]:         out.eset <- BisqueRNA::SeuratToExpressionSet(seurat, 
R[write to console]:             delim, idx, version = "v3")
R[write to console]:         out.eset@featureData@data$GeneID <- featureNames(out.eset)
R[write to console]:         out.eset@phenoData@data$cluster <- as.character(out.eset@phenoData@data$cellType)
R[write to console]:         varMetadata(out.eset) <- data.frame(labelDescription = colnames(out.eset@phenoData@data), 
R[write to console]:           

Selection: 2


From cffi callback <function _consoleread at 0x136f3bcb0>:
Traceback (most recent call last):
  File "/Users/mqp/miniconda3/envs/scanpy/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 884, in _input_request
    ident, reply = self.session.recv(self.stdin_socket, 0)
  File "/Users/mqp/miniconda3/envs/scanpy/lib/python3.7/site-packages/jupyter_client/session.py", line 813, in recv
    msg_list = socket.recv_multipart(mode, copy=copy)
  File "/Users/mqp/miniconda3/envs/scanpy/lib/python3.7/site-packages/zmq/sugar/socket.py", line 475, in recv_multipart
    parts = [self.recv(flags, copy=copy, track=track)]
  File "zmq/backend/cython/socket.pyx", line 791, in zmq.backend.cython.socket.Socket.recv
  File "zmq/backend/cython/socket.pyx", line 827, in zmq.backend.cython.socket.Socket.recv
  File "zmq/backend/cython/socket.pyx", line 186, in zmq.backend.cython.socket._recv_copy
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrup

### CRC dataset - subset only tumour patients

In [4]:
adata_crc.obs.head()

,CELL,CONDITION,Patient,Tissue,Sample,Cell_type,Cell_subtype,percent_mito,n_counts,n_genes,...,MHumanCD45p_scseqCMs6_Eff,MkHumanCD45p_scseqCMs6_Eff,MHumanCD45p_scseqCMs6_Endo,MkHumanCD45p_scseqCMs6_Endo,clusterID,cell_names,cell_group,scell_group,sscell_group,celltype_dream
index,,,,,,,,,,,,,,,,,,,,,
SMC01-T_AAACCTGCATACGCCG,SMC01-T_AAACCTGCATACGCCG,Tumor,SMC01,Tumor,SMC01-T,Epithelial cells,CMS2,0.0,33782.0,4782,...,-0.184670,0.250,-0.048650,0.076923,C17.nCD45.Epi...nNa.nCC.nCy.nCh.nAc,C17Epi,Epi,Epi,Epi,epithelial.cells
SMC01-T_AAACCTGGTCGCATAT,SMC01-T_AAACCTGGTCGCATAT,Tumor,SMC01,Tumor,SMC01-T,Epithelial cells,CMS2,0.0,31032.0,5160,...,-0.276030,0.125,-0.047704,0.076923,C17.nCD45.Epi...nNa.nCC.nCy.nCh.nAc,C17Epi,Epi,Epi,Epi,epithelial.cells
SMC01-T_AAACCTGTCCCTTGCA,SMC01-T_AAACCTGTCCCTTGCA,Tumor,SMC01,Tumor,SMC01-T,Epithelial cells,CMS2,0.0,6092.0,1677,...,-0.122945,0.125,-0.033656,0.000000,C17.nCD45.Epi...nNa.nCC.nCy.nCh.nAc,C17Epi,Epi,Epi,Epi,epithelial.cells
SMC01-T_AAACGGGAGGGAAACA,SMC01-T_AAACGGGAGGGAAACA,Tumor,SMC01,Tumor,SMC01-T,Epithelial cells,CMS2,0.0,3413.0,1193,...,-0.261535,0.000,-0.029198,0.000000,C17.nCD45.Epi...nNa.nCC.nCy.nCh.nAc,C17Epi,Epi,Epi,Epi,epithelial.cells
SMC01-T_AAACGGGGTATAGGTA,SMC01-T_AAACGGGGTATAGGTA,Tumor,SMC01,Tumor,SMC01-T,Epithelial cells,CMS2,0.0,20416.0,3843,...,-0.375577,0.000,-0.036710,0.076923,C17.nCD45.Epi...nNa.nCC.nCy.nCh.nAc,C17Epi,Epi,Epi,Epi,epithelial.cells


In [10]:
t_idx = adata_crc.obs.loc[adata_crc.obs['CONDITION']=='Tumor'].index
crc_tumor = adata_crc[t_idx].copy()

In [ ]:
# check if patient index separable using '-': for BisqueRNA::SeuratToExpressionset conversion
idx = crc_tumor.obs.index
subjects = [i.split('-')[0] for i in idx]
set(subjects)

In [15]:
# set main X to raw.X

obs = crc_tumor.obs
var = crc_tumor.raw.var
uns = crc_tumor.uns
raw = crc_tumor.raw

crc_tumor_raw = anndata.AnnData(raw.X, obs=obs, var=var, uns=uns, raw=raw)
crc_tumor_raw.write(os.path.join('./', 'crc_tumor_raw.h5ad'))

# export annot as converting to R object will replace them with factors
crc_tumor_raw_annot = crc_tumor_raw.obs['celltype_dream']
crc_tumor_raw_annot.to_csv('./crc_tumor_annot.csv', index=False, header=False)

convert to R obj

In [16]:
import anndata2ri
anndata2ri.activate()

In [17]:
%load_ext rpy2.ipython

In [18]:
%%R -i crc_tumor_raw
saveRDS(crc_tumor_raw, 'crc_tumor_raw_sce.RDS')

### Subset datasets using Geosketch

SCDC too slow with full datasets. Aim for max 8000 cells in total

In [ ]:
def geosketch_subsample(adata, N=4000, filename='adata_geosketch.h5ad', column='<last_column>', raw=True):
    
    if (column == '<last_column>'):
        column = adata.obs.columns[-1]
    print('Initiating Geosketch. This process may take a while for large datasets.')
    if raw:
        E = adata.raw.X.toarray() # convert from sparse to ndarray
        sketch_index = gs(E, N, replace=False, verbose=True)
        X_sketch = E[sketch_index]
        var = adata.raw.var
        raw_dat = None
        
    else:
        sketch_index = gs(adata.X, N, replace=False, verbose=True)
        X_sketch = adata.X[sketch_index]
        var = adata.var
        raw_dat = adata.raw[sketch_index]
        
    obs = adata.obs.iloc[sketch_index]
    uns = adata.uns.copy()
    rmkeys = ['neighbors', 'pca', 'rank_genes_groups'] # remove these entries from adata.uns as they cause issues with geosketching
    for key in rmkeys:
        uns.pop(key, None)

    
    adata_sub = anndata.AnnData(X_sketch, obs=obs, var=var, uns=uns, raw=raw_dat)
    adata_sub.write(os.path.join('./', filename))

    anno = adata_sub.obs[column]
    anno.to_csv(os.path.join('./', filename.split('.')[0] + '_scanno.csv'), index=False, header=False)
    
    return adata_sub


In [ ]:
geosketch_subsample(adata_brca, filename='brca_raw_geosketch.h5ad', raw=True)

In [ ]:
geosketch_subsample(adata_martin, filename='martin_raw_geosketch.h5ad', raw=True)

In [ ]:
adata_crc_geo = geosketch_subsample(adata_crc, filename='crc_raw_geosketch.h5ad', raw=False)

keep_keys = ['louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_groups']
ks = adata_martin.uns.copy().keys()
for k in ks:
    if k not in keep_keys:
        adata_martin.uns.pop(k, None)
        
ks = adata_brca.uns.copy().keys()
for k in ks:
    if k not in keep_keys:
        adata_brca.uns.pop(k, None)

### brca select only tumour patients & non sparse data

In [ ]:
# only select tumour samples
idx = adata_brca.obs.index
new_idx = []

for i, value in enumerate(idx):
    if 'TUMOR' in value:
        new_idx.append(i)
        


In [ ]:
brca_tumour = adata_brca[new_idx].copy()

In [ ]:
idx = brca_tumour.obs.index
new_idx = [i.replace('_','-') for i in idx]
brca_tumour.obs.index = new_idx

In [ ]:
# export raw
obs = brca_tumour.obs
var = brca_tumour.raw.var
uns = brca_tumour.uns
raw = brca_tumour.raw

brca_tumour_raw = anndata.AnnData(raw.X, obs=obs, var=var, uns=uns, raw=raw)
brca_tumour_raw.write(os.path.join('./', 'brca_raw_tumour_only.h5ad'))

In [ ]:
brca_tumour_raw_annot = brca_tumour_raw.obs['celltype_dream']
brca_tumour_raw_annot.to_csv('./brca_tumor_annot.csv', index=False, header=False)

In [ ]:
brca_tumour_raw.obs.head()

### Subset adata.raw


In [ ]:
idx = adata_martin_sub.obs.index
geo_idx = adata_martin.obs.index.isin(idx)
rawX = adata_martin.raw.X[geo_idx]

In [ ]:
# dirty way
obs = adata_martin_sub.obs
var = adata_martin_sub.raw.var
uns = adata_martin_sub.uns
raw = adata_martin_sub.raw

adata_martin_raw = anndata.AnnData(raw.X, obs=obs, var=var, uns=uns, raw=raw)
adata_martin_raw.write('martin_geosketch_raw.h5ad')

In [ ]:
obs = adata_brca_sub.obs
var = adata_brca_sub.raw.var
uns = adata_brca_sub.uns
raw = adata_brca_sub.raw

adata_brca_raw = anndata.AnnData(raw.X, obs=obs, var=var, uns=uns, raw=raw)
adata_brca_raw.write('brca_geosketch_raw.h5ad')

In [ ]:

obs = adata_crc_geo.obs
var = adata_crc_geo.raw.var
uns = adata_crc_geo.uns
raw = adata_crc_geo.raw

adata_crc_raw = anndata.AnnData(raw.X, obs=obs, var=var, uns=uns, raw=raw)
adata_crc_raw.write('crc_raw_geosketch.h5ad')

set raw.X as X for full non-geoskchetch dataset

In [ ]:
adata_brca
adata_martin

In [ ]:
obs = adata_brca.obs
var = adata_brca.raw.var
uns = adata_brca.uns
raw = adata_brca.raw

adata_brca_raw = anndata.AnnData(raw.X, obs=obs, var=var, uns=uns, raw=raw)
adata_brca_raw.write('brca_raw.h5ad')

In [ ]:
obs = adata_martin.obs
var = adata_martin.raw.var
uns = adata_martin.uns
raw = adata_martin.raw

adata_martin_raw = anndata.AnnData(raw.X, obs=obs, var=var, uns=uns, raw=raw)
adata_martin_raw.write('martin_raw.h5ad')

### Martin data - are there too many missing cell types?

In [ ]:
adata_martin.obs.loc[adata_martin.obs['Subject'] == 'pat. 5']['celltype_dream'].unique()

In [ ]:
adata_martin_geo_raw.obs.loc[adata_martin_geo_raw.obs['Subject'] == 'pat. 5']['celltype_dream'].unique()

In [ ]:
adata_brca.obs.head()

In [ ]:
adata_brca.obs.loc[adata_brca.obs['ID'] == 'BC01_BLOOD1']['celltype_dream'].unique()

In [ ]:
adata_brca_geo_raw.obs.loc[adata_brca_geo_raw.obs['ID'] == 'BC01_BLOOD1']['celltype_dream'].unique()

### Fix martin index - for BisqueRNA::SeuratToExpressionSet()
currently splitting with this index gives two labels per individual as follows

In [ ]:
idx5 = adata_martin_geo_raw.obs.loc[adata_martin_geo_raw.obs['Subject']=='pat. 5'].index
ids5 = []
for i in idx5:
    ids5.append(i.split('.')[0])
len(set(ids5))
set(ids5)

thus fix index

In [ ]:
idx = adata_martin.obs.index
idx_new = []
for i, val in enumerate(idx):
    out = adata_martin.obs['Subject'][i].replace(" ", "").replace('.','') + '_' + val.split('_')[1]
    idx_new.append(out)

In [ ]:
adata_martin.obs.index = idx_new

In [ ]:
adata_martin.obs.head()

### geosketch output analysis

check if all subjects still represented, likewise for cell types

In [ ]:
# 11 samples in orig ds
adata_martin.obs['Subject'].values.unique()

In [ ]:
# 50 samples in original ds
adata_brca.obs['ID'].values.unique()

In [ ]:
adata_brca_geo_raw = sc.read('./brca_geosketch_raw.h5ad')
adata_martin_geo_raw = sc.read('./martin_geosketch_raw.h5ad')

In [ ]:
# still 11 subjects
adata_martin_geo.obs['Subject'].values.unique()

In [ ]:
# still 50 subjects
adata_brca_geo_raw.obs['ID'].values.unique()

now check cell types

In [ ]:
# geosketch missing 'naive.CD8.T.cells'
x = adata_brca.obs['celltype_dream'].values.unique().tolist()
y = adata_brca_geo_raw.obs['celltype_dream'].values.unique().tolist()
set(x) - set(y)

In [ ]:
# geosketch missing 'naive.CD8.T.cells'
x = adata_martin.obs['celltype_dream'].values.unique().tolist()
y = adata_martin_geo_raw.obs['celltype_dream'].values.unique().tolist()
set(x) - set(y)

### Convert to SingleCellExperiment R Object

saves anndata as SCE object. Postprocess in R

based on tutorial here: https://github.com/LuckyMD/Code_snippets/blob/master/Seurat_to_anndata.ipynb

In [ ]:
import anndata2ri
anndata2ri.activate()

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R -i brca_tumour_raw
saveRDS(brca_tumour_raw, 'brca_raw_tumour_only_sce.RDS')

In [ ]:
%%R -i adata_brca_raw
saveRDS(adata_brca_raw, 'brca_raw_sce.RDS')

In [ ]:
# do brca
adata_brca_sub = sc.read('brca_geosketch.h5ad')

In [ ]:
%%R -i adata_brca_sub
saveRDS(adata_brca_sub, 'brca_sce.RDS')

In [ ]:
%%R -i adata_martin_raw
saveRDS(adata_martin_raw, 'martin_raw_sce.RDS')

In [ ]:
%%R -i adata_brca_raw
saveRDS(adata_brca_raw, 'brca_raw_sce.RDS')

In [ ]:
#%%R -i adata_martin
#library(Seurat)
#martin.seurat <- as.Seurat(adata_martin)
#saveRDS(martin.seurat, 'martin_seurat.RDS')

### Notes converting SCE to eset in R

run from cmd line: `env R_MAX_VSIZE=100Gb R`

### Fix issues loading martin ds in R ReadH5AD: 
Error in file[["obs"]][] : object of type 'environment' is not subsettable

Note: these were not needed. the issue was with a bug in AnnData new version: https://github.com/satijalab/seurat/issues/2485

In [ ]:
adata_martin_sub.obs.head()
# thus in R: 
# out.eset <- BisqueRNA::SeuratToExpressionSet(seurat, delimiter='_', position=2, version = 'v3')

In [ ]:
adata_brca_sub.obs.head()


In [ ]:
adata_martin.obs.columns

In [ ]:
rmkeys = ['neighbors', 'pca', 'rank_genes_groups', 'celltype_dream_colors']
for key in rmkeys:
    adata_martin.uns.pop(key, None)

In [ ]:
adata_martin.uns.keys()

In [ ]:
keep_keys = ['louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_groups']
ks = adata_martin.uns.copy().keys()
for k in ks:
    if k not in keep_keys:
        adata_martin.uns.pop(k, None)

In [ ]:
adata_martin.write(filename='./martin_anndata07rc1.h5ad')

### Check SCDC error
Error in y[y < q15] <- q15[y < q15] :
NAs are not allowed in subscripted assignments
In addition: There were 50 or more warnings (use warnings() to see the first 50)

In FUN(newX[, i], ...) : no non-missing arguments to max; returning -Inf

In [ ]:
# check if NaNs in datasets
np.isnan(adata_martin.X).any()

In [ ]:
np.isnan(adata_brca.X).any()

### reduce number of features

scdc too slow

In [ ]:
# remove others?
adata_brca.obs.loc[adata_brca.obs['celltype_dream']=='others']

In [ ]:
adata_martin.obs.loc[adata_martin.obs['celltype_dream']=='others']

### determine missing celltypes for each dataset 

In [ ]:
ct_b = adata_brca.obs['celltype_dream'].values.unique().tolist()

In [ ]:
ct_m = adata_martin.obs['celltype_dream'].values.unique().tolist()

In [ ]:
ct_s = adata_smillie.obs['celltype_dream'].values.unique().tolist()

In [ ]:
# dream celltypes 
ct = """memory.B.cells
naive.B.cells
memory.CD4.T.cells
naive.CD4.T.cells
regulatory.T.cells
memory.CD8.T.cells
naive.CD8.T.cells
NK.cells
neutrophils
monocytes
myeloid.dendritic.cells
macrophages
fibroblasts
endothelial.cells""".split()

In [ ]:
# intersection dream x brca
set(ct) & set(ct_b)

In [ ]:
list(set(ct) - set(ct_m))

In [ ]:
# brca dataset has all required celltypes
list(set(ct) - set(ct_b))

In [ ]:
# intersect brca x martin
set(ct_b) & set(ct_m)

In [ ]:
# brca x smillie
set(ct_b) & set(ct_s)

In [ ]:
print(','.join("'{0}'".format(w) for w in ct))

'memory.B.cells','naive.B.cells','memory.CD4.T.cells','naive.CD4.T.cells','regulatory.T.cells','memory.CD8.T.cells','naive.CD8.T.cells','NK.cells','neutrophils','monocytes','myeloid.dendritic.cells','macrophages','fibroblasts','endothelial.cells'